<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Visualização Interativa de Dados
Caderno de **Aula**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Google Data Studio;</li>
  <li>Dados;</li>
  <li>Relatório.</li>
</ol>

---

# **Aulas**

## 1\. Google Data Studio

### **1.1. Inteligência de negócio** 

> Inteligência de negócio, do inglês *Business Intelligenc*e (BI), é uma especialidade da análise de dados que busca facilitar o acesso a informação para tomada de decisão através de ferramentas de visualização interativa de dados.

Um dos objetivos da área é a definição (e exposição) de **indicadores chaves de desempenho** (*key performance indicato*r ou KPI) e facilitação do uso da **análise exploratória de dados** (*exploratory data analysis* ou EDA) de forma interativa para usuário de negócio (geralmente não técnicos) através do uso de ferramentas de BI.

### **1.2. Ferramentas de BI** 

As ferramentas líderes de mercado são o [Tableau](https://www.tableau.com) e o [Microsoft Power BI](https://powerbi.microsoft.com/), ambas pagas com uma limitada versão gratuita. Neste curso utilizaremos o [Google Data Studio](https://datastudio.google.com/) que apresenta uma versão completa e gratuita.

### **1.3. Termos técnicos** 

Diferentes ferramentas de visualização interativa de dados utilizam um conjunto de termos técnicos comun para construção de painéis de dados: 

 - **Relatório** (*report*): o painel de dados (*dashboard*);
 - **Fonte de dados** (*data source*): conjunto de dados usado nas visualizações de dados do painel;
 - **Dimensões** (*dimensions*): coluna do conjunto de dados;
 - **Métricas** (*metrics*): operação de agregação aplicada a uma ou mais dimensões (soma, média, etc.).

### **1.4. Google Data Studio** 

O Data Studio é uma ferramenta *online* e gratuita de visualização interativa de dados da Google. Para ter acesso a ferramenta, basta ter um conta Google e clicar neste [link](https://datastudio.google.com/). As duas principais ferramentas são:

 - **Reports** para criar painéis de dados;
 - **Data source** para criar conectar painéis a fontes de dados.

## 2\. Dados

Neste módulo, vamos analisar dados sobre o a população mundial, em milhares de habitantes, entre os anos de 1950 e 2021. O conjunto de dados está neste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/world-population.csv) e foi inspirado no conjunto de dados (compilados pelas nações unidas, sobre o tema do Kaggle, presente neste [link](https://www.kaggle.com/ahmethoso/wpp-population-by-age-and-sex?select=PopulationByAgeSex.csv). O painel de dados é o destino final dos dados, logo, todo o processo de manipulação deve ser feito a priori.

In [ ]:
import pandas as pd
from datetime import datetime

### **2.1. Extração** 

Nesta etapa, vamos extrair os dados da fonte de dados original e avaliar a sua qualidade.

In [ ]:
!wget -q 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/world-population.csv' -O 'world_population.csv'

In [ ]:
data = pd.read_csv('world_population.csv', sep=',', parse_dates=[2], infer_datetime_format=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

### **2.2. Transformação** 

Vamos então aplicar uma série de transformações para limpar e pre-processar a base de dados.

 - Descartar a coluna `id`:

In [ ]:
data = data.drop(['id'], axis=1)

 - Renomear a coluna `location` para `country` e `time` para `date`:

In [ ]:
data = data.rename(columns={'location': 'country', 'time': 'date'})

 - Ajustar a coluna `date` para o [formato](https://support.google.com/datastudio/answer/6401549?hl=en#zippy=%2Cin-this-article) do Google Data Studio:

In [ ]:
data['date'] = data['date'].apply(lambda num: datetime.strftime(num, '%b %d, %Y'))

 - Adicionar valores relativos as colunas de população parcial:

In [ ]:
data['child_perc'] = 100 * data['child'] / data['total']
data['adolescent_perc'] = 100 * data['adolescent'] / data['total']
data['adult_perc'] = 100 * data['adult'] / data['total']
data['old_perc'] = 100 * data['old'] / data['total']

In [ ]:
data['male_perc'] = 100 * data['male'] / data['total']
data['female_perc'] = 100 * data['female'] / data['total']

 - Arredondar todas as colunas de população para duas casas decimais:

In [ ]:
for col in data.columns:
  if col not in ['date', 'country']:
    data[col] = data[col].apply(lambda num: round(num, 2))

 - Reorganizar as colunas:

In [ ]:
data = data[['date', 'country', 'total', 'male', 'male_perc', 'female',	'female_perc', 'child', 'child_perc', 'adolescent', 'adolescent_perc', 'adult', 'adult_perc', 'old', 'old_perc']]

Vamos conferir o resultado final.

In [ ]:
data.head()

### **2.3. Carregamento** 

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
data.to_csv('./world-population-wrangled.csv', sep=',', index=False)

## 3\. Relatório

Como dito na aula anterior, o relátorio ou painel de dados é o destino final dos dados. É atavés que pessoas interessadas no acompanhamento da atividade exercída vão se informar, em geral, pessoas menos técnicas e mais orientadas a negócio como analistas, gerentes e executivos. Portanto, um bom design de um painel (cores, tipografia, escolha e organização de elementos, etc.) é fundamental para transmitir a informação com qualidade.

### **3.1. EDA & KPIs** 

Os paineis de dados buscam entregar duas funcionalidades para o usuário final:

1. o consumo de **indicadores chaves de desempenho** (*key performance indicator* ou KPI) consolidados;
1. a capacidade de praticar a **análise exploratória de dados** (*exploratory data analysis* ou EDA) interativa.

Em geral, o primeiro são métricas consolidadas que fazem parte do acompanhamento da atividade exercida, costumam aparecer no topo do painel. O restante do espaço útil é preenchido por elementos interativos (gráficos, mapas, etc.) que permitem o usuário a entender melhor uma dimensão específica.

### **3.2. Elementos** 

As ferramentes de visualização de dados interativa possuem diversos elementos para a composição de um relatório como ferramentas gerais de edição de texto e imagens, gráficos, tabelas, cartões, etc. Em geral, a manipulação dos elementos se resume a interação entre três sub elementos:

 - Dimensões (colunas);
 - Métricas (agregações);
 - Filtros.

### **3.3. Compartilhamento** 

Por fim, painéis de dados do Google Data Studio podem ser compartilhados na internet, assim como os cadernos do Kaggle.